# This notebook contains all 3 parts of the assigment

## Part 1


In [1]:
#Import Libraries
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import requests # Library for web scraping
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import ssl
import csv




In [2]:
# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

print('SSL certificate errors ignored.')

SSL certificate errors ignored.


In [3]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(source, 'lxml')

print('soup ready')

soup ready


In [4]:
#Create table
table = soup.find('table',{'class':'wikitable sortable'})

In [5]:
#table rows
table_rows = table.find_all('tr')

In [6]:
#Build data set
data = []
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')])

df = pd.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighbourhood'])
df = df[~df['PostalCode'].isnull()]  # to filter out bad rows

In [7]:
df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 287 entries, 1 to 287
Data columns (total 3 columns):
PostalCode       287 non-null object
Borough          287 non-null object
Neighbourhood    287 non-null object
dtypes: object(3)
memory usage: 9.0+ KB


In [8]:
df.shape

(287, 3)

In [9]:
df.head(5)

,PostalCode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


In [10]:
#Drop unassigned borough
df.drop(df[df['Borough']=="Not assigned"].index,axis=0, inplace=True)
df.head(12)

,PostalCode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Downtown Toronto,Queen's Park
10,M9A,Queen's Park,Not assigned
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern
14,M3B,North York,Don Mills North


In [11]:
df1 = df.reset_index()
df1.head()

,index,PostalCode,Borough,Neighbourhood
0,3,M3A,North York,Parkwoods
1,4,M4A,North York,Victoria Village
2,5,M5A,Downtown Toronto,Harbourfront
3,6,M6A,North York,Lawrence Heights
4,7,M6A,North York,Lawrence Manor


In [12]:
#Group by postal code
df2= df1.groupby('PostalCode').agg(lambda x: ','.join(x))

In [13]:
df.head(12)

,PostalCode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Downtown Toronto,Queen's Park
10,M9A,Queen's Park,Not assigned
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern
14,M3B,North York,Don Mills North


In [14]:
#Borough=Neiborhood if needed
df2.loc[df2['Neighbourhood']=="Not assigned",'Neighbourhood']=df2.loc[df2['Neighbourhood']=="Not assigned",'Borough']
df2.head(12)

,Borough,Neighbourhood
PostalCode,,
M1B,"Scarborough,Scarborough","Rouge,Malvern"
M1C,"Scarborough,Scarborough,Scarborough","Highland Creek,Rouge Hill,Port Union"
M1E,"Scarborough,Scarborough,Scarborough","Guildwood,Morningside,West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae
M1J,Scarborough,Scarborough Village
M1K,"Scarborough,Scarborough,Scarborough","East Birchmount Park,Ionview,Kennedy Park"
M1L,"Scarborough,Scarborough,Scarborough","Clairlea,Golden Mile,Oakridge"
M1M,"Scarborough,Scarborough,Scarborough","Cliffcrest,Cliffside,Scarborough Village West"


In [15]:
df3 = df2.reset_index()

In [16]:
df3.head(12)

,PostalCode,Borough,Neighbourhood
0,M1B,"Scarborough,Scarborough","Rouge,Malvern"
1,M1C,"Scarborough,Scarborough,Scarborough","Highland Creek,Rouge Hill,Port Union"
2,M1E,"Scarborough,Scarborough,Scarborough","Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,"Scarborough,Scarborough,Scarborough","East Birchmount Park,Ionview,Kennedy Park"
7,M1L,"Scarborough,Scarborough,Scarborough","Clairlea,Golden Mile,Oakridge"
8,M1M,"Scarborough,Scarborough,Scarborough","Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,"Scarborough,Scarborough","Birch Cliff,Cliffside West"


In [17]:
#clean it up
df3['Borough']= df3['Borough'].str.replace('nan|[{}\s]','').str.split(',').apply(set).str.join(',').str.strip(',').str.replace(",{2,}",",")

## ----------> First data frame wanted 

In [18]:
df3.head(12)

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


In [19]:
df3.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 3 columns):
PostalCode       103 non-null object
Borough          103 non-null object
Neighbourhood    103 non-null object
dtypes: object(3)
memory usage: 2.5+ KB


In [20]:
df3.shape

(103, 3)

# Part 2

In [23]:
#!conda install -c conda-forge geocoder

loc = pd.read_csv('http://cocl.us/Geospatial_data')
loc.head()

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geocoder-1.38.1            |             py_1          53 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    ratelim-0.1.6              |             py_2           6 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.4 MB

The following NEW packages will be INSTALLED:

    geocoder:        1.38.1-py_1       conda-forge
    ratelim:         0.1.6-py_2        conda-forge

The following packages will be UPDATED:

    

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [24]:
import geocoder

def get_latlng(postal_code):
    # initialize variable to None
    lat_lng_coords = None
    # loop until getting the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    return lat_lng_coords
    
get_latlng('M4G')

[43.70949500000006, -79.36398897099997]

In [26]:
postal_codes = df3['PostalCode']    
coords = [ get_latlng(postal_code) for postal_code in postal_codes.tolist() ]


## -------------> 2nd data frame wanted

In [27]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
df3['Latitude'] = df_coords['Latitude']
df3['Longitude'] = df_coords['Longitude']
df3.head(12)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.785665,-79.158725
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.765815,-79.175193
3,M1G,Scarborough,Woburn,43.768369,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440
5,M1J,Scarborough,Scarborough Village,43.743125,-79.231750
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.726276,-79.263625
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.713054,-79.285055
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.724235,-79.227925
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.696770,-79.259967


In [28]:
df[df.PostalCode == 'M5V']

,PostalCode,Borough,Neighbourhood
216,M5V,Downtown Toronto,CN Tower
217,M5V,Downtown Toronto,Bathurst Quay
218,M5V,Downtown Toronto,Island airport
219,M5V,Downtown Toronto,Harbourfront West
220,M5V,Downtown Toronto,King and Spadina
221,M5V,Downtown Toronto,Railway Lands
222,M5V,Downtown Toronto,South Niagara


In [29]:
df3.head(10)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.785665,-79.158725
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.765815,-79.175193
3,M1G,Scarborough,Woburn,43.768369,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440
5,M1J,Scarborough,Scarborough Village,43.743125,-79.231750
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.726276,-79.263625
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.713054,-79.285055
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.724235,-79.227925
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.696770,-79.259967


# Part 3

In [32]:
import requests
from bs4 import BeautifulSoup as bs
import re

import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          92 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.50-py_0   conda-forge
    geopy:         1.21.0-py_0 conda-forge


geopy-1.21.0         | 58 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


## Mapping of the differents neighborhoods
### Note that one marker sometimes gather different neighborhoods due to similar postal codes

In [101]:
address = 'Toronto'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df3['Latitude'], df3['Longitude'], df3['Borough'], df3['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


The geograpical coordinate of Toronto are 43.653963, -79.387207.


## Vizualising Downtown Toronto boroughs

In [102]:
DT = df3[df3['Borough'] == 'DowntownToronto'].reset_index(drop=True)
address1 = 'Downtown Toronto,Toronto'

geolocator1 = Nominatim()
location1 = geolocator1.geocode(address1)
latitude1 = location1.latitude
longitude1 = location1.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude1, longitude1))

DT.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:4: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.


The geograpical coordinate of Downtown Toronto are 43.6541737, -79.38081164513409.


,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M4W,DowntownToronto,Rosedale,43.682205,-79.377945
1,M4X,DowntownToronto,"Cabbagetown,St. James Town",43.668160,-79.366602
2,M4Y,DowntownToronto,Church and Wellesley,43.666585,-79.381302
3,M5A,DowntownToronto,Harbourfront,43.650295,-79.359166
4,M5B,DowntownToronto,"Ryerson,Garden District",43.657363,-79.378180


In [36]:
map_DT = folium.Map(location=[latitude1, longitude1], zoom_start=11)

# add markers to map
for lat, lng, label in zip(DT['Latitude'], DT['Longitude'], DT['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_DT)  
    
map_DT

## Looking up the venues in Downtown Toronto

In [ ]:
neighborhood_latitude = DT.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = DT.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = DT.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

LIMIT = 100
radius = 500

In [131]:
# The code was removed by Watson Studio for sharing.

In [103]:

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude1, longitude1, VERSION, radius, LIMIT)

Latitude and longitude values of Rosedale are 43.68220500000007, -79.37794519699997.


In [104]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [105]:
results = requests.get(url).json()

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(12)


,name,categories,lat,lng
0,UNIQLO ユニクロ,Clothing Store,43.655910,-79.380641
1,Elgin And Winter Garden Theatres,Theater,43.653394,-79.378507
2,Indigo,Bookstore,43.653515,-79.380696
3,Ed Mirvish Theatre,Theater,43.655102,-79.379768
4,CF Toronto Eaton Centre,Shopping Mall,43.654083,-79.380716
5,SEPHORA,Cosmetics Shop,43.653688,-79.380120
6,Yonge-Dundas Square,Plaza,43.656054,-79.380495
7,Eggspectation Bell Trinity Square,Breakfast Spot,43.653144,-79.381980
8,JOEY Eaton Centre,Restaurant,43.655404,-79.381929
9,LUSH,Cosmetics Shop,43.653557,-79.380400


In [106]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [129]:
DT_venues = getNearbyVenues(names=DT['Neighbourhood'],
                                   latitudes=DT['Latitude'],
                                   longitudes=DT['Longitude']
                                  )
DT_venues.head()

Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie
Queen's Park


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,43.682205,-79.377945,Summerhill Market,43.686265,-79.375458,Grocery Store
1,Rosedale,43.682205,-79.377945,Rosedale Park,43.682328,-79.378934,Playground
2,Rosedale,43.682205,-79.377945,Whitney Park,43.682036,-79.373788,Park
3,Rosedale,43.682205,-79.377945,Scoops Convenience Boutique,43.686148,-79.375828,Candy Store
4,"Cabbagetown,St. James Town",43.668160,-79.366602,F'Amelia,43.667536,-79.368613,Italian Restaurant


In [108]:
DT.head(3)
print(DT_venues.groupby('Neighborhood').count()[:4])

                                                    Neighborhood Latitude  \
Neighborhood                                                                
Adelaide,King,Richmond                                                100   
Berczy Park                                                            61   
CN Tower,Bathurst Quay,Island airport,Harbourfr...                     71   
Cabbagetown,St. James Town                                             38   

                                                    Neighborhood Longitude  \
Neighborhood                                                                 
Adelaide,King,Richmond                                                 100   
Berczy Park                                                             61   
CN Tower,Bathurst Quay,Island airport,Harbourfr...                      71   
Cabbagetown,St. James Town                                              38   

                                                    Venue  Venue Lat

## Determine number of venues by category for each neighborhood
## Integrate the results to the existing dataset 

In [ ]:
# one hot encoding
DT_onehot = pd.get_dummies(DT_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
DT_onehot['Neighborhood'] = DT_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [DT_onehot.columns[-1]] + list(DT_onehot.columns[:-1])
DT_onehot = DT_onehot[fixed_columns]

DT_grouped = DT_onehot.groupby('Neighborhood').mean().reset_index()

DT_grouped.head()

In [110]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [119]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = DT_grouped['Neighborhood']

for ind in np.arange(DT_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(DT_grouped.iloc[ind, :], num_top_venues)
    
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Hotel,Steakhouse,Breakfast Spot,Japanese Restaurant,Burger Joint,Gym,Bar,Bakery
1,Berczy Park,Coffee Shop,Cocktail Bar,Breakfast Spot,Seafood Restaurant,Farmers Market,Steakhouse,Beer Bar,Restaurant,Café,Bakery
2,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Coffee Shop,Italian Restaurant,Bar,Café,Sandwich Place,Gym / Fitness Center,Intersection,Pub,Park,Bakery
3,"Cabbagetown,St. James Town",Coffee Shop,Restaurant,Italian Restaurant,Pizza Place,Café,Bakery,Liquor Store,Breakfast Spot,Snack Place,Japanese Restaurant
4,Central Bay Street,Coffee Shop,Clothing Store,Bakery,Ice Cream Shop,Bubble Tea Shop,Spa,Electronics Store,Sandwich Place,Bookstore,Gym / Fitness Center


## Cluster the neighborhood

In [112]:
DT_data = DT.drop(16)
# set number of clusters
kclusters = 7

DT_grouped_clustering = DT_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(DT_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:]

DT_grouped_clustering.head()

,Yoga Studio,Adult Boutique,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Quad,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,Historic Site,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pier,Pizza Place,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recreation Center,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Steakhouse,Strip Club,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0.000000,0.0,0.0,0.030000,0.0,0.010000,0.0,0.030000,0.0,0.000000,0.0,0.000000,0.030000,0.000000,0.030000,0.000000,0.010000,0.000000,0.0,0.000000,0.0,0.020000,0.01,0.030000,0.000000,0.000000,0.01,0.030000,0.01,0.000000,0.060000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.070000,0.0,0.0,0.000000,0.0,0.0,0.01,0.000000,0.000000,0.020000,0.000000,0.000000,0.000000,0.0,0.020000,0.010000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.010000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.01,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.030000,0.0,0.000000,0.01,0.01,0.01,0.000000,0.010000,0.000000,0.030000,0.010000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.050000,0.000000,0.010000,0.000000,0.000000,0.000000,0.010000,0.030000,0.000000,0.000000,0.000000,0.0,0.010000,0.000000,0.000000,0.010000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.01,0.000000,0.000000,0.000000,0.010000,0.0,0.01,0.000000,0.01,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.020000,0.000000,0.010000,0.000000,0.0,0.0,0.010000,0.000000,0.0,0.000000,0.030000,0.0,0.0,0.01,0.01,0.010000,0.0,0.020000,0.000000,0.000000,0.010000,0.000000,0.0,0.000000,0.010000,0.000000,0.040000,0.0,0.020000,0.01,0.000000,0.000000,0.000000,0.0,0.010000,0.0,0.010000,0.010000,0.0,0.000000,0.0,0.000000,0.0

In [120]:
DT_merged.head()
#neighborhoods_venues_sorted.shape
#kmeans.labels_.shape

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M4W,DowntownToronto,Rosedale,43.682205,-79.377945
1,M4X,DowntownToronto,"Cabbagetown,St. James Town",43.668160,-79.366602
2,M4Y,DowntownToronto,Church and Wellesley,43.666585,-79.381302
3,M5A,DowntownToronto,Harbourfront,43.650295,-79.359166
4,M5B,DowntownToronto,"Ryerson,Garden District",43.657363,-79.378180


In [128]:
DT_merged = DT_data
#DT_merged.head()

# add clustering labels
DT_merged['Cluster Labels'] = kmeans.labels_[0:18]

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
DT_merged = DT_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')


,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4W,DowntownToronto,Rosedale,43.682205,-79.377945,4,Playground,Grocery Store,Park,Candy Store,Dim Sum Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
1,M4X,DowntownToronto,"Cabbagetown,St. James Town",43.668160,-79.366602,4,Coffee Shop,Restaurant,Italian Restaurant,Pizza Place,Café,Bakery,Liquor Store,Breakfast Spot,Snack Place,Japanese Restaurant
2,M4Y,DowntownToronto,Church and Wellesley,43.666585,-79.381302,0,Coffee Shop,Gay Bar,Japanese Restaurant,Restaurant,Sushi Restaurant,Café,Pub,Fast Food Restaurant,Men's Store,Gastropub
3,M5A,DowntownToronto,Harbourfront,43.650295,-79.359166,4,Coffee Shop,Bakery,Boat or Ferry,Theater,Café,Dessert Shop,Mexican Restaurant,Chocolate Shop,Pub,Hotel
4,M5B,DowntownToronto,"Ryerson,Garden District",43.657363,-79.378180,0,Coffee Shop,Clothing Store,Japanese Restaurant,Cosmetics Shop,Café,Tea Room,Italian Restaurant,Lingerie Store,Middle Eastern Restaurant,Hotel


## Vizualise the clusters in Downton Toronto 
### Similiar neighborhood, regarding their venues, will be displayed with the same color

In [127]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(DT_merged['Latitude'], DT_merged['Longitude'], DT_merged['Neighbourhood'], DT_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters